# 01 — From Structs to Classes: The Mental Shift 🧠

## 1. The C View vs. The C++ View

In C, you view a struct as a POD (Plain Old Data). It is just a block of memory. If you want to do something with it, you pass a pointer to a standalone function.

```c
// The C Way
struct Vector3 {
    float x, y, z;
};

void normalize(struct Vector3* v) {
    // Manually manipulate data via pointer
}
```

In C++, a class (or struct) binds data and logic together.

### ❓ The Million Dollar Question: Is there overhead?
No. (Mostly).

A class with 3 floats takes up exactly 12 bytes (assuming 4-byte floats), same as in C. Member functions are not stored inside the object. They are stored in the text (code) segment, just like C functions.

When you call `obj.method()`, the compiler secretly passes `&obj` as a hidden argument called `this`.


---


## 2. Encapsulation: Why private?

In C, we trust the programmer not to set `array_len = -5`. In C++, we don't.

We use `private` to enforce invariants. An invariant is a rule that must always be true for the object to be valid (e.g., "Size cannot be negative", "Pointer cannot be null").


In [ ]:
#include <iostream>

class BankAccount {
private:
    // Data is hidden. You cannot touch this directly from main().
    double balance;

public:
    // Constructor (Initializes the object)
    BankAccount(double initial_deposit) {
        if (initial_deposit < 0) balance = 0;
        else balance = initial_deposit;
    }

    // Public Interface
    void deposit(double amount) {
        if (amount > 0) balance += amount;
    }

    double get_balance() const {
        return balance;
    }
};

BankAccount account(100.0);
// account.balance = -5000; // COMPILER ERROR! Protected.

account.deposit(50.0);
std::cout << "Balance: " << account.get_balance() << std::endl;
return 0;


---


## 3. The Lifecycle: Constructors vs. `init()`

In C, you often write:

```c
struct System s;
system_init(&s);   // Don't forget this!
// ... use s ...
system_cleanup(&s); // Don't forget this either!
```

In C++, the Constructor guarantees initialization, and the Destructor guarantees cleanup. You cannot create an uninitialized class object if you write a constructor.


In [ ]:
#include <iostream>

class FileHandler {
public:
    FileHandler() {
        std::cout << "[Resource Acquired] File opened." << std::endl;
    }

    ~FileHandler() {
        std::cout << "[Resource Released] File closed automatically." << std::endl;
    }
};

std::cout << "Entering scope..." << std::endl;
{
    FileHandler fh; // Constructor runs HERE
    std::cout << "Working with file..." << std::endl;
} // Destructor runs HERE automatically (Scope ends)

std::cout << "Exiting main..." << std::endl;
return 0;

---


## 4. ⚠️ The `memset` Trap (CRITICAL)

As a C programmer, you love `memset(&obj, 0, sizeof(obj))` to zero out data.

NEVER do this to a non-POD C++ object.

Why?
1. Internal pointers: `std::string` or `std::vector` manage internal pointers to heap memory. If you `memset` them to 0, you overwrite those pointers. You leak memory and corrupt the object.
2. V-Tables: If a class has virtual functions, the object contains a hidden pointer (vptr) to a dispatch table. `memset` zeros this out, causing the program to crash immediately upon calling a method.


In [ ]:
#include <iostream>
#include <string>
#include <cstring> // for memset

struct C_Style_Struct {
    int id;
    float value;
};

struct Cpp_Class {
    std::string name;
    int id;
};

// --- SAFE ---
C_Style_Struct c_obj;
std::memset(&c_obj, 0, sizeof(c_obj)); // Totally fine for PODs
std::cout << "memset on POD: Safe." << std::endl;

// --- DANGEROUS ---
Cpp_Class cpp_obj;
cpp_obj.name = "Hello";

// UNCOMMENTING THE LINE BELOW WILL BREAK THE STRING
// std::memset(&cpp_obj, 0, sizeof(cpp_obj));

// In C++, we use Constructors or assignment to reset objects.
cpp_obj = Cpp_Class{}; // Reassign a fresh empty object

std::cout << "Reset C++ object safely." << std::endl;
return 0;